# Dataset 4: Análisis exploratorio

Todo lo acontecido a nivel mundial en los últimos meses afecta al sector agroalimentario global, pudiendo verse muy influenciado todo el comercio internacional de frutas y hortalizas. Esto afecta a su vez al mercado español, lo que hace que sea importante tener en cuenta datos de importaciones y exportaciones de España, pudiendo comparar periodos anteriores a 2020 con lo sucedido durante este año. El cuarto dataset contiene la serie histórica mensual de comercio exterior de España con el resto de países de Europa desde 2018 de diferentes frutas y hortalizas, presentando las siguientes variables tanto para importaciones como para exportaciones por producto y origen/destino:

    - Cantidad de producto en valor monetario (EUROS)
    - Cantidad de producto en peso (100 KG)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns;
sns.set()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline


#Esta celda de código permite reducir el tamaño de los márgenes laterales  y aprovechar mejor el tamaño de la ventana.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# OBEJTIVO: Analizar el comportamiento del mercado español de frutas y hortalizas durante el periodo de pandemia (1/3/20 - 30/11/20).

En primer lugar, leemos el dataset. Este dataset muestra un problema, y es que cada producto tiene 4 filas duplicadas, una para cada tipo de `FLOW`, `INDICATORS` y `Value`. Como vemos, otro problema es que la variable `Value` presenta filas con ':' como valor, que tendríamos que sustituir por NaN.

In [2]:
df4 = pd.read_csv('data/Dataset4.- Comercio Exterior de España.txt', sep='|', na_values=':')
df4

,PERIOD,REPORTER,PARTNER,PRODUCT,FLOW,INDICATORS,Value
0,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",IMPORT,VALUE_IN_EUROS,NaN
1,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",IMPORT,QUANTITY_IN_100KG,NaN
2,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",EXPORT,VALUE_IN_EUROS,NaN
3,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",EXPORT,QUANTITY_IN_100KG,NaN
4,Feb. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",IMPORT,VALUE_IN_EUROS,NaN
...,...,...,...,...,...,...,...
360971,Oct. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",EXPORT,QUANTITY_IN_100KG,NaN
360972,Nov. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",IMPORT,VALUE_IN_EUROS,NaN
360973,Nov. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",IMPORT,QUANTITY_IN_100KG,NaN
360974,Nov. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",EXPORT,VALUE_IN_EUROS,NaN


La variable `Value` es la única que tiene valores perdidos, que son más de la mitad de los casos. Pirmero la convertiremos a numérica para ver qué podemos hacer con ella. Algunos valores de esta variable tiene espacios, por lo que tendremos que quitarselos para poderla convertir.

In [3]:
np.sum(df4.isna())

PERIOD             0
REPORTER           0
PARTNER            0
PRODUCT            0
FLOW               0
INDICATORS         0
Value         223408
dtype: int64

In [4]:
df4['Value'] = df4['Value'].str.replace(' ', '', regex=False).astype(float)

In [5]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360976 entries, 0 to 360975
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   PERIOD      360976 non-null  object 
 1   REPORTER    360976 non-null  object 
 2   PARTNER     360976 non-null  object 
 3   PRODUCT     360976 non-null  object 
 4   FLOW        360976 non-null  object 
 5   INDICATORS  360976 non-null  object 
 6   Value       137568 non-null  float64
dtypes: float64(1), object(6)
memory usage: 11.0+ MB


# Corregir nombres de Países

In [6]:
dataframe_names = df4['REPORTER'].unique().tolist()
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
# replace data frame names by json names
df4.replace(dict(zip(dataframe_names, eu_countries)), inplace=True)
df4

,PERIOD,REPORTER,PARTNER,PRODUCT,FLOW,INDICATORS,Value
0,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",IMPORT,VALUE_IN_EUROS,NaN
1,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",IMPORT,QUANTITY_IN_100KG,NaN
2,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",EXPORT,VALUE_IN_EUROS,NaN
3,Jan. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",EXPORT,QUANTITY_IN_100KG,NaN
4,Feb. 2018,Austria,ES,"Bananas, fresh (excl. plantains)(2012-2500)",IMPORT,VALUE_IN_EUROS,NaN
...,...,...,...,...,...,...,...
360971,Oct. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",EXPORT,QUANTITY_IN_100KG,NaN
360972,Nov. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",IMPORT,VALUE_IN_EUROS,NaN
360973,Nov. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",IMPORT,QUANTITY_IN_100KG,NaN
360974,Nov. 2020,Sweden,ES,"White and red cabbages, fresh or chilled(1988-...",EXPORT,VALUE_IN_EUROS,NaN


# Filtrar por países de la UE

Como el dataset es bastante grande, hemos decidido trabajar solo con los países de la Unión Europea.

In [7]:
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
df4 = df4[df4['REPORTER'].isin(eu_countries)]
df4.shape

(360976, 7)

# Añadir columna fecha

Para poder hacer esto, primero debemos quedarnos solo con las filas del dataset que no tengan periodos problemáticos, como son 'Jan. - Dec. 2018'.

In [8]:
df4 = df4[~df4['PERIOD'].str.contains("-")]
df4.shape

(341264, 7)

In [9]:
df4[['M', 'Y']] = df4['PERIOD'].str.split(". ", expand=True)
months_dict = {'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April', 'Jun': 'June', 'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}
df4['M'] = df4['M'].replace(months_dict)
df4['DATE'] = pd.to_datetime(df4['Y'] + df4['M'], format='%Y%B')

In [10]:
df4['Y'] = pd.to_numeric(df4['Y'])
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341264 entries, 0 to 360975
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PERIOD      341264 non-null  object        
 1   REPORTER    341264 non-null  object        
 2   PARTNER     341264 non-null  object        
 3   PRODUCT     341264 non-null  object        
 4   FLOW        341264 non-null  object        
 5   INDICATORS  341264 non-null  object        
 6   Value       127124 non-null  float64       
 7   M           341264 non-null  object        
 8   Y           341264 non-null  int64         
 9   DATE        341264 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 19.5+ MB


## Eliminar los datos de diciembre de 2020 (faltan datos de varios productos)

In [16]:
df4 = df4[df4['PERIOD'] != 'Dec. 2020']

### Profiling de las variables

In [17]:
profile = ProfileReport(df4, title='Comercio esterior de España')
profile

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Exportar datos

In [18]:
df4.to_csv('data/Dataset4.txt', sep='|', index=False)